In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ai agent trainning/train/reward')

import sys
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

Mounted at /content/drive


In [ ]:
!pip install -q bitsandbytes accelerate peft trl==0.15.2 datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import yaml

with open('../configs/base.yaml', 'r') as file:
    configs = yaml.safe_load(file)

with open('../configs/reward.yaml', 'r') as file:
    configs.update(yaml.safe_load(file))

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model

In [ ]:
from peft import AutoPeftModelForCausalLM, AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer
# backbone
backbone = AutoPeftModelForCausalLM.from_pretrained(
    configs['llm_backbone'],
    load_in_4bit = configs['model']['load_in_4bit'],
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(configs['llm_backbone'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [ ]:
from peft import get_peft_model, PeftModel
from reward import RewardModel

In [ ]:
reward_model = RewardModel(backbone)
peft_reward_model = PeftModel.from_pretrained(reward_model, configs['pretrain_model'])
peft_reward_model.reward_head = torch.load(configs['pretrain_model']+"/reward_head.pt")

In [ ]:
import json

with open("../../data/reward/health-care-reward.json", "r") as f:
    data = json.load(f)

In [ ]:
import random
idx = random.randint(0, len(data))

# question = """Tôi 28 tuổi, từng điều trị lao phổi cách đây 6 năm và sau đó bị tăng cân nhanh, kinh nguyệt không đều; hiện tôi nặng 75 kg và đang cố gắng thụ thai dù các xét nghiệm đều bình thường — liệu tôi còn cơ hội có con không?"""
# helpful = """Dưới góc độ bác sĩ, tôi xin trình bày lại như sau:
# 1. Việc tăng cân đáng kể — từ 6 kg đến 20 kg — có thể là nguyên nhân gây rối loạn kinh nguyệt và ảnh hưởng đến khả năng thụ thai.
# 2. Tiền sử điều trị lao phổi, cũng như các vấn đề huyết học (như tán huyết hoặc bất thường bạch cầu), có thể khiến cơ thể rối loạn chức năng ở mức độ tế bào.
# 3. Tuy nhiên, các xét nghiệm máu hiện tại của bạn đều trong giới hạn bình thường.
# 4. Tôi không thấy có dấu hiệu đáng lo ngại vào lúc này; bạn hoàn toàn có thể mang thai. Tôi khuyên bạn nên thực hiện đầy đủ các kiểm tra tiền sản để đảm bảo an toàn trước khi bắt đầu quá trình thụ thai.
# Bạn có muốn mình gợi ý các kiểm tra cần thiết trước khi mang thai không?
# """
# bad = """Vui lòng b. T. Th. Bạn đang sử dụng thuốc chống tát dạ dày và các thuốc giải phình của miệng. Tôi đã kể trên Chat Doctor. Hy vọng có thể thấy rằng bạn sẽ hoàn thành điều này. B. Hy vọng mọi thứ sẽ được bình thường. Tôi hiểu rằng bạn không nên quá khóc nang. Mua thuốc điều tra. Khẩn liễ. Bạn sẽ bị phát triển lên và bạn đã biết tôi đang giải đáp được sự vây vào trong bệnh của bạn. Vui lòng B. C. H. Bạn có thể thêm câu hỏi. Bạn có thể được làm lại tốt hơn nếu bạn đã theo đuổi nghi ngờ có thể bị nhiễm lao về mức độ. Cáu nào sẽ giúp bạn. L. H. Th"""

question = data[idx]['question']
helpful = data[idx]['chosen']
bad = data[idx]['rejected']

inputs = tokenizer(helpful, return_tensors='pt').to(device)
good_score = peft_reward_model(**inputs).logits[0].cpu().detach()

inputs = tokenizer(bad, return_tensors='pt').to(device)
bad_score = peft_reward_model(**inputs).logits[0].cpu().detach()

print(good_score,'||', bad_score)
print(good_score > bad_score)

tensor([0.0544], dtype=torch.bfloat16) || tensor([-0.1562], dtype=torch.bfloat16)
tensor([True])
